In [1]:
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)  # reproducibility

sample = " if you want you"
idx2char = list(set(sample))  # index -> char
char2idx = {c: i for i, c in enumerate(idx2char)}  # char -> idex
print("idx2char", idx2char)
print("char2idx", char2idx)

idx2char ['t', 'i', 'f', 'w', 'a', 'n', ' ', 'y', 'u', 'o']
char2idx {'t': 0, 'i': 1, 'f': 2, 'w': 3, 'a': 4, 'n': 5, ' ': 6, 'y': 7, 'u': 8, 'o': 9}


In [2]:
# hyper parameters
dic_size = len(char2idx)     # RNN input size (one hot size)
hidden_size = len(char2idx)  # RNN output size
num_classes = len(char2idx)  # final output size (RNN or softmax, etc.)
batch_size = 1               # one sample data, one batch
sequence_length = len(sample) - 1  # number of lstm rollings (unit #)
learning_rate = 0.1

sample_idx = [char2idx[c] for c in sample]  # char to index
x_data = [sample_idx[:-1]]   # X data sample (0 ~ n-1) hello: hell
y_data = [sample_idx[1:]]    # Y label sample (1 ~ n) hello: ello

X = tf.placeholder(tf.int32, [None, sequence_length])  # X data
Y = tf.placeholder(tf.int32, [None, sequence_length])  # Y label

In [3]:
# num_classes: 몇 개의 one_hot으로 만들어줄 것인지
# 유일한(unique) 문자 집합인 idx2char 사이즈와 동일
x_one_hot = tf.one_hot(X, num_classes)  # one hot: 1 -> 0 1 0 0 0 0 0 0 0 0

cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=initial_state, dtype=tf.float32)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [4]:
# FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)

# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

In [5]:
weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

In [6]:
prediction = tf.argmax(outputs, axis=2)

In [7]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(50):
        l, _ = sess.run([loss, train], feed_dict={X: x_data, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_data})

        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]
        print(i, "loss:", l, "Prediction:", ''.join(result_str))

0 loss: 2.2986755 Prediction: o    u  u     u
1 loss: 2.142747 Prediction: o   ou uuuuuuou
2 loss: 1.9777694 Prediction: o   ouuuuuuuuou
3 loss: 1.7914783 Prediction: y   ou uu   you
4 loss: 1.5178113 Prediction: yf you  a t you
5 loss: 1.2456594 Prediction: yf you aant you
6 loss: 0.9785164 Prediction: yf you  ant you
7 loss: 0.731504 Prediction: yf you want you
8 loss: 0.53080505 Prediction: yf you want you
9 loss: 0.37683412 Prediction: if you want you
10 loss: 0.25416905 Prediction: if you want you
11 loss: 0.16722944 Prediction: if you want you
12 loss: 0.111258306 Prediction: if you want you
13 loss: 0.07425931 Prediction: if you want you
14 loss: 0.049364645 Prediction: if you want you
15 loss: 0.033410687 Prediction: if you want you
16 loss: 0.023288494 Prediction: if you want you
17 loss: 0.016653432 Prediction: if you want you
18 loss: 0.012179709 Prediction: if you want you
19 loss: 0.009172107 Prediction: if you want you
20 loss: 0.007192319 Prediction: if you want you
21 l